# ConvLSTM for Patches
Trains a convolutional LSTM. The model input is a stack of 64x64 patches of the total area of interest, and the model output is a 64x64 patch with the predicted land type at the next time step. Requires about 12 GB RAM to run.

In [ ]:
%matplotlib inline
import keras_core as keras
import tensorflow as tf
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
SEED = 42
rng = np.random.default_rng(SEED)

Define parameters for patch size.

In [ ]:
PATCH_SIZE = 64  # Size of each patch in pixels
OVERLAP_SIZE = 64  # Number of pixels to advance before accessing the next patch
MAX_EMPTY_RATIO = 0.4  # Maximum percent of pixels in the image that can be zero
TIME_STEPS = 5  # Number of time steps

## Dataset Generation
Read all input files and stack them on top of each other to create a large numpy array.

In [ ]:
images = []
for f in glob('data/CONUS20*_ClipAOI*.tif'):
    with rasterio.open(f) as ds:
        data = ds.read(1)
        images.append(data)
images = np.array(images)
n_times = images.shape[0]
images.shape

Compute a 2D prefix sum array for the entire large image. When passing patches to the model during training, we want to exclude patches where the entire image or the majority of pixels are out of bounds (zero). Calculating the prefix sum array for the entire large image will allow fast querying of the number of zero pixels in any given patch.

In [ ]:
image = images[0, :, :]
prefix = np.zeros_like(image, dtype=np.uint32)
prefix[image == 0] = 1
prefix = np.cumsum(np.cumsum(prefix, axis=0, dtype=np.uint32), axis=1, dtype=np.uint32)
prefix.shape

In [ ]:
def get_zero_pixels(i, j):
    """Calculates the number of zero pixels in the patch with corner at (i, j)."""
    zeros = prefix[i + PATCH_SIZE - 1, j + PATCH_SIZE - 1]
    if i > 0 and j > 0:
        zeros += prefix[i - 1, j - 1]
    if i > 0:
        zeros -= prefix[i - 1, j + PATCH_SIZE - 1]
    if j > 0:
        zeros -= prefix[i + PATCH_SIZE - 1, j - 1]
    return zeros

Determine the possible categories and normalize them to integer values starting at 0.

In [ ]:
categories, counts = np.unique(image, return_counts=True)
n_categories = categories.shape[0]
category_map = {categories[i]: i for i in range(n_categories)}
percents = counts / image.size * 100
del image
plt.bar(list(map(str, categories)), percents)
plt.title('Land Type Distribution')
plt.ylabel('Percent')
plt.show()
category_map

Using the prefix sums array, find the indices of every patch in the dataset that lies in the area of interest.

In [ ]:
indices = []
for i in range(0, images.shape[1] - PATCH_SIZE, OVERLAP_SIZE):
    for j in range(0, images.shape[2] - PATCH_SIZE, OVERLAP_SIZE):
        zeros = get_zero_pixels(i, j)
        if zeros >= PATCH_SIZE * PATCH_SIZE * MAX_EMPTY_RATIO:
            continue
        indices.append((i, j))
del prefix  # Not needed anymore
indices = np.array(indices)
rng.shuffle(indices)
indices.shape

## Training the Model
Define training parameters.

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1
VAL_SPLIT = 0.1

In [ ]:
val_size = int(indices.shape[0] * VAL_SPLIT)
val_indices = indices[:val_size, :]
train_indices = indices[val_size:, :]
train_indices.shape, val_indices.shape

Build the dataset using the list of patch indices. The full dataset is generated in-place using a generator function because it would be too large to fit in memory.

In [ ]:
def one_hot(x):
    encoded = np.zeros(x.shape + (n_categories,), dtype=np.uint8)
    for category, index in category_map.items():
        category_mask = (x == category)
        encoded[category_mask, index] = 1
    return encoded

def get_data(indices):
    sparse_encoder = np.vectorize(lambda x: category_map[x], otypes=[np.uint8])
    for i, j in indices:
        for k in range(n_times - TIME_STEPS):
            x = one_hot(images[k:k + TIME_STEPS, i:i + PATCH_SIZE, j:j + PATCH_SIZE])
            y = sparse_encoder(images[k + TIME_STEPS, i:i + PATCH_SIZE, j:j + PATCH_SIZE])
            yield x, y

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: get_data(train_indices),
    output_signature=(
        tf.TensorSpec(shape=(TIME_STEPS, PATCH_SIZE, PATCH_SIZE, n_categories), dtype=tf.float32),
        tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE), dtype=tf.uint8)
    )
)
train_ds = train_ds.apply(tf.data.experimental.assert_cardinality(train_indices.shape[0] * (n_times - TIME_STEPS)))
train_ds = train_ds.batch(BATCH_SIZE)

val_ds = tf.data.Dataset.from_generator(
    lambda: get_data(val_indices),
    output_signature=(
        tf.TensorSpec(shape=(TIME_STEPS, PATCH_SIZE, PATCH_SIZE, n_categories), dtype=tf.float32),
        tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE), dtype=tf.uint8)
    )
)
val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(val_indices.shape[0] * (n_times - TIME_STEPS)))
val_ds = val_ds.batch(BATCH_SIZE)

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(TIME_STEPS, PATCH_SIZE, PATCH_SIZE, n_categories)),
    keras.layers.ConvLSTM2D(64, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.ConvLSTM2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(n_categories, kernel_size=(3, 3), padding='same', activation='softmax')
])
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name='acc')
    ]
)
model.summary()

In [ ]:
model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)